# Many to many

In [12]:
from sqlalchemy.orm import DeclarativeBase # Декларативный, базовый класс
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Session # для создания записей в бд
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, text, select
from typing import List
import sqlite3
from sqlalchemy import func
from sqlalchemy import Column, Table
from datetime import datetime

Tutorial from SQLAlchemy

In [21]:
class Base(DeclarativeBase):
  pass


class Order(Base):
  __tablename__ = "orders"

  id: Mapped[int] = mapped_column(primary_key=True)
  promocode: Mapped[str | None]
  created_at: Mapped[datetime] = mapped_column(
      server_default=func.now(), # что будет на стороне сервера при создании это записи
      default=datetime.utcnow # что будет по умолчанию
    )
  products: Mapped[list["Product"]] = relationship(secondary="order_product_associstion",
                                                   back_populates="orders") # ссылаемся на промежуточную таблицу


order_product_associstion_table = Table(
    "order_product_associstion",  # имя таблицы
    Base.metadata, # мета-данные
    # колонки в любом количестве (какая таблица, а какая справа - неважно)
    # Здесь мы создаем колонки
    Column("order_id", ForeignKey("orders.id"), primary_key=True), # primary_key - один товар в корзине может быть только один раз
    Column("product_id", ForeignKey("products.id"), primary_key=True)
)


class Product(Base):
  __tablename__ = "products"

  id: Mapped[int] = mapped_column(primary_key=True)
  name: Mapped[str]
  description: Mapped[str]
  price: Mapped[int]

  orders: Mapped[list["Product"]] = relationship(secondary="order_product_associstion",
                                                   back_populates="products") # ссылаемся на промежуточную таблицу

Another example

In [22]:
class Base(DeclarativeBase):
  pass


class Order(Base):
  __tablename__ = "orders"

  id: Mapped[int] = mapped_column(primary_key=True)
  promocode: Mapped[str | None]
  created_at: Mapped[datetime] = mapped_column(
      server_default=func.now(), # что будет на стороне сервера при создании это записи
      default=datetime.utcnow # что будет по умолчанию
    )
  order_relation: Mapped[List['Product']] = relationship(
      back_populates="orders",
      secondary="orders_products" #указываем промежуточную таблицу
  )


class Product(Base):
  __tablename__ = "products"

  id: Mapped[int] = mapped_column(primary_key=True)
  name: Mapped[str]
  description: Mapped[str]
  price: Mapped[int]
  product_relation: Mapped[List['Order']] = relationship(
      back_populates="products",
      secondary="orders_products" #указываем промежуточную таблицу
  )

# Промежуточная таблица
class ProductOrders(Base):
  __tablename__ = "orders_products"

  # два первичных ключа
  order_id: Mapped[int] = mapped_column(
      ForeignKey("orders.id", ondelete="CASCADE"), # настройка каскадного удаления, чтобы при удаении заказа, были удалены все продукты, которые в нем содержатся
      primary_key=True
  )

  product_id: Mapped[int] = mapped_column(
      ForeignKey("products.id", ondelete="CASCADE"),
      primary_key=True
  )

In [23]:
# Создаем модели в базе данных
engine = create_engine(
    url="sqlite:///./db.sqlite3"
)
Base.metadata.create_all(bind=engine)